In [1]:
import os
import csv
import numpy as np
from sklearn import model_selection
from sklearn import metrics
# from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
import pickle
import matplotlib.pyplot as plt

In [2]:
code_path = os.getcwd()
input_dir = os.path.join(code_path,'radarScenes_v5_Nmin')
print(input_dir)

/home/user/deng/catkin_deng/src/track/DBSCAN_Train/radarScenes_v5_Nmin


In [11]:
def csv_read():
  print("Reading directory:",input_dir)
  files = os.listdir(input_dir)
  files.sort(key=lambda x:x[:11])
  print('Training Sequence:',len(files))
  Nmin = []
  TrainX = []
  for filename in files:
    csv_path = os.path.join(input_dir,filename)
    with open(csv_path,newline='') as csv_file:
      rows = csv.reader(csv_file)
      first = True
      for row in rows:
        if first:
          first = False
          continue
        if str(row).find('nan') != -1 or str(row).find('inf') != -1:
          continue
        Nmin.append(int(row[2]))
        input_row = []
        input_row.append(float(row[4])/int(row[11])) # vel slot 0
        input_row.append(float(row[5])/int(row[11])) # vel slot 1
        input_row.append(float(row[6])/int(row[11])) # vel slot 2
        input_row.append(float(row[7])/int(row[11])) # vel slot 3
        input_row.append(float(row[8])/int(row[11])) # vel slot 4
        input_row.append(float(row[9])/int(row[11])) # vel slot 5
        input_row.append(int(row[11])) # current scan num

        TrainX.append(input_row)
  NminMatrix = np.asarray(Nmin)
  NminMatrix.reshape(len(Nmin),1)
  # Nmin_min = np.min(NminMatrix)
  # Nmin_max = np.max(NminMatrix)
  # NminMatrix = (NminMatrix-Nmin_min)/(Nmin_max-Nmin_min)
  print('Target:',NminMatrix[0])
  print('Input:',TrainX[0])
  TrainMatrix = np.asarray(TrainX)
  X_train, X_test, Y_train, Y_test = model_selection.train_test_split(np.array(TrainMatrix), np.array(NminMatrix), test_size=0.2)
  print('Training Element: vel slot 0~5, current scan num; Training Target: Nmin in iou 0.3')
  return X_train, X_test, Y_train, Y_test, TrainMatrix, NminMatrix

In [12]:
X_train, X_test, Y_train, Y_test, X_total, Y_total = csv_read()

Reading directory: /home/user/deng/catkin_deng/src/track/DBSCAN_Train/radarScenes_v5_Nmin
Training Sequence: 112
Target: 2
Input: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 300]
Training Element: vel slot 0~5, current scan num; Training Target: Nmin in iou 0.3


In [5]:
def save_model(model,name):
    out_path = os.path.join(code_path,name+'.sav')
    pickle.dump(model,open(out_path,'wb'))

In [6]:
def evaluate_model(model,X_test,Y_test):
    predict_y_test = model.predict(X_test)
    print('Test r-score: ',metrics.r2_score(Y_test,predict_y_test))
    print('Test mean: ',metrics.mean_squared_error(Y_test,predict_y_test))
    print(Y_test)
    print(predict_y_test)

In [7]:
def load_model(name):
    load_path = os.path.join(code_path,name+'.sav')
    return pickle.load(open(load_path, 'rb'))

In [23]:
training_model_name = 'Nmin_MLP_model_logistic_v3'

In [ ]:
print('Training Net'+training_model_name+'!')
model = MLPRegressor(hidden_layer_sizes=200, max_iter=300, activation='logistic')
model.fit(X_train,Y_train)
save_model(model, training_model_name)

In [ ]:
print(model.get_params())
evaluate_model(model,X_test,Y_test)
plt.plot(model.loss_curve_)

In [ ]:
for i in range(50):
    model.partial_fit(X_train,Y_train)
    print('End training iter',i)
    save_model(model,training_model_name)

In [ ]:
#v2 result
print(model.get_params())
evaluate_model(model,X_test,Y_test)
plt.plot(model.loss_curve_)

In [ ]:
# v1 result
print(model.get_params())
evaluate_model(model,X_test,Y_test)
plt.plot(model.loss_curve_)

In [ ]:
intercepts = np.array(model.intercepts_)
print(intercepts.shape)
print(repr(intercepts[0]))
print(repr(intercepts[1]))

In [ ]:
coef = np.array(model.coefs_)
print(coef[0].shape)
print(coef[1].shape)
print(repr(coef[1].reshape(200)))
# print(model.coefs_)
for i in range(7):
    print(repr(coef[0][i]))


In [13]:
from sklearn.linear_model import LinearRegression
# v2: use Nmin in iou>=0.5, v3: use Nmin in iou>=0.3
training_model_name_2 = 'Nmin_Linear_model_v3'
model2 = LinearRegression().fit(X_total, Y_total)
save_model(model2,training_model_name_2)

In [14]:
print(model2.get_params())
evaluate_model(model2,X_test,Y_test)

{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': False}
Test r-score:  0.16316787452293802
Test mean:  3.510938840546214
[4 2 2 ... 5 3 9]
[4.49625754 4.10634395 2.95595983 ... 3.2554395  4.11950623 7.82684467]


In [15]:
linear_coef = model2.coef_
print(repr(linear_coef))
print(repr(model2.intercept_))

array([-7.64444524e+00,  1.78340216e+01,  2.02924245e+00, -6.12427851e-01,
       -1.78487167e+01,  6.24232575e+00,  2.09028092e-03])
9.781014951107098
